In [4]:
# To ignore warinings
import warnings
warnings.filterwarnings('ignore')

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [6]:
# Reading the dataset
df = pd.read_csv('/kaggle/input/nepali-news-classification-set/full_cleaned.csv')

In [7]:
df.head()

,title,news_stopwords,news_no_stopwords,category
0,निखिल उप्रेतीको भैरव फिल्मले अमेरिकामा रहेका न...,अमेरिकाका नेपालीको प्रतिक्रिया लिने इच्छा व्यक...,अमेरिकाका नेपालीको प्रतिक्रिया लिने इच्छा व्यक...,मनोरञ्जन
1,सुशील कोइरालाको निधनपछि चौरासी बाले खोले यस्ता...,झण्डै बर्षअघि सुशील कोइरालाले प्रधानमन्त्रीको ...,झण्डै बर्षअघि सुशील कोइरालाले प्रधानमन्त्रीको ...,मनोरञ्जन
2,लिटल प्रिन्स एण्ड प्रिन्सेसको ग्रान्ड फिनाले,ग्ल्यामरस नेपालले सुरुङ्गामा लिटल प्रिन्स एन्ड...,ग्ल्यामरस नेपालले सुरुङ्गामा लिटल प्रिन्स एन्ड...,मनोरञ्जन
3,अमेरिकामा सबैभन्दा धेरै कमाउने सिईओ बने पिचाई ...,गूगलका सिईओ सुन्दर पिचाई अमेरिकामा सबैभन्दा धे...,गूगलका सिईओ सुन्दर पिचाई अमेरिकामा सबैभन्दा कम...,मनोरञ्जन
4,ज्योती मगरको धमाका दोहोरीमा र्याप,दोहोरीमा र्याप बोलको गीतको भिडियो सार्वजनिक भए...,दोहोरीमा र्याप बोलको गीतको भिडियो सार्वजनिक चर...,मनोरञ्जन


In [8]:
df.drop(columns=["title", "news_stopwords"], inplace=True)

In [9]:
category_mapping = {
    "मनोरञ्जन": 0,
#     "देश/प्रदेश": 1,
    "अर्थ / वाणिज्य": 1,
    "विश्व": 2,
    "खेलकुद": 3,
    "समाज": 4,
    "राजनीति": 5,
    "विज्ञान र प्रविधि": 6,
    "स्वास्थ्य": 7,
} 

In [10]:
CONSTANTS = {}
CONSTANTS["total_categories"] = 8

In [11]:
def random_undersampling(data, random_state):
    if len(data) > 25000:
        return data.sample(25000, random_state=random_state)
    return data

In [12]:
# Drop Duplicates
df.drop_duplicates(["news_no_stopwords"], inplace=True)# Remove rows with null values
df.dropna(inplace=True, axis=0)
# Remove sikshya category
df = df[df["category"] != "शिक्षा"]
# Remove desh pradesh category
df = df[df["category"] != "देश/प्रदेश"]
# Map catgeories to numbers
df["category"] = df["category"].map(category_mapping)
# Compute length of each news articles
df["length"] = df["news_no_stopwords"].apply(lambda x: len(x.split()))
# Remove all news with <30 words
df = df[df["length"] >= 30]
# Remove all news with length > 300
CONSTANTS["max_news_length"]= 300
df["news_no_stopwords"] = df["news_no_stopwords"].apply(lambda x: " ".join(x.split()[:CONSTANTS["max_news_length"]]))
# Perform random undersampling
df_balanced = pd.DataFrame(columns=['news_no_stopwords', 'category'])
for i in range(CONSTANTS["total_categories"]):
    res = random_undersampling(df[df["category"] == i], i * np.random.randint(100))
    if df_balanced.empty == True:
        df_balanced = res
    else:
        df_balanced = pd.concat([df_balanced, res], ignore_index=True)

# Shuffle the dataset
for i in range(20):
    df_balanced = df_balanced.sample(frac=1)

In [13]:
df_balanced.head()

,news_no_stopwords,category,length
40426,नेपालमा दीगो विकास लक्षित जिम्मेवार व्यावसायिक...,1,70
103245,समुदायस्तरमै कोरोना संक्रमण फैलिएपछि प्रदेश सं...,4,334
89388,कोरोना भाइरसको महामारीको कारण भनिएको साफ च्याम...,3,52
74263,संयुक्तराज्य अमेरिकामा संघीय कानुन बमोजिम व्यक...,2,124
37554,अर्थमन्त्री डाप्रकाशशरण महतले व्यक्ति व्यवसायी...,1,148


In [14]:
len(df_balanced)

194517

In [15]:
# Train test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_balanced["news_no_stopwords"], df_balanced["category"], test_size=0.10, random_state=34, stratify=df_balanced["category"])

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, confusion_matrix, classification_report

In [17]:
tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=52000)

In [18]:
X_train_tfidf = tfidf.fit_transform(X_train)

In [19]:
X_test_tfidf = tfidf.transform(X_test)

In [20]:
mnb = MultinomialNB()
mnb.fit(X_train_tfidf, y_train)
mnb_preds = mnb.predict(X_test_tfidf)

In [21]:
print(classification_report(y_test, mnb_preds))

              precision    recall  f1-score   support

           0       0.92      0.93      0.92      2500
           1       0.75      0.77      0.76      2500
           2       0.86      0.84      0.85      2500
           3       0.98      0.91      0.95      2500
           4       0.71      0.64      0.67      2500
           5       0.79      0.89      0.84      2500
           6       0.84      0.83      0.83      2303
           7       0.79      0.83      0.81      2149

    accuracy                           0.83     19452
   macro avg       0.83      0.83      0.83     19452
weighted avg       0.83      0.83      0.83     19452



In [31]:
from joblib import dump, load
dump(mnb, 'mnb_nepali_text_classifier.joblib') 

['mnb_nepali_text_classifier.joblib']